In [ ]:
DISABLE_INTERNET = True
INFERENCE = True

# Kaggle or Colab
import sys
import os
if 'kaggle_web_client' in sys.modules:
    # Do something
    pass
elif 'google.colab' in sys.modules:
    # Do something
    from google.colab import drive
    drive.mount("/content/drive")

    comp_name_official = "chaii-hindi-and-tamil-question-answering"
    comp_name_local = "Kaggle-chaii"

    !pip install --upgrade --force-reinstall --no-deps kaggle
    import json
    f = open("/content/drive/MyDrive/colab_notebooks/kaggle/kaggle.json", "r")
    json_data = json.load(f)
    os.environ["KAGGLE_USERNAME"] = json_data["username"]
    os.environ["KAGGLE_KEY"] = json_data["key"]

    %cd /content/drive/MyDrive/colab_notebooks/kaggle/{comp_name_local}/notebooks

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
if 'kaggle_web_client' in sys.modules:
    # Do something
    pass
elif 'google.colab' in sys.modules:
    !pip install transformers

In [ ]:
import re

In [ ]:
from transformers import AdamW
from transformers import get_scheduler
#from transformers import BertTokenizer, BertModel

In [ ]:
from transformers import BertTokenizerFast

In [ ]:
from transformers import BertForQuestionAnswering

In [ ]:
if DISABLE_INTERNET:
    model_path = "../input/localnb001-export-transformers"
    tokenizer = BertTokenizerFast.from_pretrained(model_path)
else:
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

In [ ]:
test = pd.read_csv("../input/chaii-hindi-and-tamil-question-answering/test.csv")
#test.head()

In [ ]:
# train["context_string_length"] = train["context"].apply(lambda x: len(x))
# train["context_token_length"] = train["context"].apply(lambda x: len(x.split()))

# plt.figure()
# col = "context_string_length"
# train[col].hist(bins=128)
# plt.xlabel(f"{col}")
# plt.title(f"Dist. of {col}");

# plt.figure()
# col = "context_token_length"
# train[col].hist(bins=128)
# plt.xlabel(f"{col}")
# plt.title(f"Dist. of {col}");
# print(f"num of obs w/ {col} being shorter than or equal to 512:\n", len(train[train[col]<=512]))
# print(f"num of obs w/ {col} being longer than 512:\n", len(train[train[col]>512]))

# plt.figure()
# train["answer_text"].apply(lambda x: len(x.split())).hist(bins=64)
# plt.xlabel("answer text length")
# plt.title("Dist. of answer text length");

**最大512長で分割**<br><br>

The hack is to split the clinical notes into multiple sub-notes such that every sub-note when concatenated with input question, gives close to 512 tokens. After concatenating the input question with sub-note, there are two possibilities: 1) if sub-note contains the ground truth answer, label the answer with its start and end position in sub-note 2) if sub-note does not contain ground truth answer, label the start and end position as 0 (points to [CLS] token).<br><br>
reference: [Adapting BERT question answering for the medical domain](https://medium.com/analytics-vidhya/adapting-bert-question-answering-for-the-medical-domain-2085ada8ceb1)

オーバーラップは60 tokens とする.

In [ ]:
# Text cleansing for context

#alphabets = "([A-Za-z])"
alphabets = "([\u0900-\u097F\u0B80-\u0BFF])" # Hindi & Tamil
prefixes = re.compile("(Mr|St|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|Mt)[.]")
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = re.compile("[.](co|net|org|io|gov|edu|us)")
etal = re.compile(r"(\bet al)[.]")
urls = re.compile("(www)[.]")
digits =  re.compile("[.]([0-9])")

def split_into_sentences(text):
    """
    This function will be applied to context in df
    ----------------------------------------------
    Args: 
        text: str
    Returns:
        sentences: List[str]
    """
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = prefixes.sub("\\1<prd>",text)
    text = websites.sub("<prd>\\1",text)
    text = urls.sub("\\1<prd>",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    text = etal.sub("\\1<prd>", text)
    text = digits.sub("<prd>\\1",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    if sentences[-1] == '':
        sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

def clean_text(txt, apply_space_norm=False, strip=False):
    """
    This function will be applied to question and answer in df
    ----------------------------------------------------------
    Args: 
        text: str
    Returns:
        text: str
    """
    txt = re.sub('\[[0-9]\]', '', txt) # remove Wikipedia's quotes
    if apply_space_norm:
        txt = re.sub('[^A-Za-z\u0900-\u097F\u0B80-\u0BFF0-9]+', ' ', str(txt).lower()) # Hindi: \u0900-\u097F, Tamil: \u0B80-\u0BFF
    if strip:
        txt = txt.strip()
    return txt


def shorten_sentences(sentences, max_length=512, overlap=60):
    """
    If a sentence is longer than `max_length`, break it into chunks of 
    length `max_length` with an overlap of length `overlap`.
    
    e.g. if the sentence has 50 tokens, max_length is 20, and overlap is 10.
    Then the first sentence will be token_i where i in [0,20)
    Second sentence will be token_i in [10,30).
    Third sentence [20, 40)
    Fourth [30, 50)
    Fifth [40, 60)
    ------------------------------------------------------------------------
    Args:
        sentences: List[str]
        max_length: int
        overlap: int
    Returns:
        shortend_sentences: List[str]
    """
    shortened_sentences = []
    for sentence in sentences:
        sentence = clean_text(sentence, apply_space_norm=True)
        words = sentence.split()
        num_words = len(words)
        if num_words > max_length:
            for start_index in range(0, num_words, max_length - overlap):
                shortened_sentences.append(' '.join(words[start_index:start_index+max_length]))
        else:
            shortened_sentences.append(sentence)
    return shortened_sentences

# refenrece:
# [1] https://stackoverflow.com/questions/41356013/how-to-detect-if-a-string-contains-hindi-devnagri-in-it-with-character-and-wor
# [2] https://en.wikipedia.org/wiki/Unicode_block
# [3] https://www.kaggle.com/nbroad/no-training-question-answering-model?scriptVersionId=66240356

In [ ]:
test["context_sentences"] = test["context"].apply(clean_text).apply(split_into_sentences).apply(shorten_sentences)

In [ ]:
test["question"] = test["question"].apply(clean_text, apply_space_norm=True, strip=True)

In [ ]:
#test["answer_text"] = test["answer_text"].apply(clean_text, apply_space_norm=True, strip=True)

In [ ]:
#test

In [ ]:
# def get_proper_answer_start(df, run_checker=False):
#     """
#     Args:
#         df: pandas.DataFrame
#         run_checker: bool --default False
#     Returns: 
#         df: pandas.DataFrame
#     """
#     df["answer_start_tw_per_sentence_s"] = 0
#     df["answer_start_tw_per_sentence_s"] = df["answer_start_tw_per_sentence_s"].astype("object")
    
#     for i,row in df.iterrows():
        
#         # Check if the answer_text is in the context sentence
#         row_context_sentences = row["context_sentences"]
#         row_answer_text = row["answer_text"]
        
#         answer_start_tw_per_sentence = []
#         for context_sentence in row_context_sentences:
#             answer_start_sw = context_sentence.find(row_answer_text)
#             if answer_start_sw==-1:
#                 answer_start_tw = answer_start_sw
#             else:
#                 answer_start_tw = len(context_sentence[:answer_start_sw+1].split()) - 1
#             answer_start_tw_per_sentence.append(answer_start_tw)
        
#         df.at[i, "answer_start_tw_per_sentence_s"] = answer_start_tw_per_sentence
#     return df

In [ ]:
#train = get_proper_answer_start(train)

In [ ]:
#train.info()

In [ ]:
#train["answer_start_tw_per_sentence_s"]

In [ ]:
#test_ids, test_contexts, test_questions = read_chaii_test(test)

In [ ]:
#assert len(test_ids) == len(test_contexts) == len(test_questions)
#print(len(test_ids), len(test_contexts), len(test_questions))

In [ ]:
# batch_size: 8 * len(test_dataloader): 21 == 168

In [ ]:
# n = 100
# test_ids[:n]

In [ ]:
# test_contexts[:n]

In [ ]:
# test_questions[:n]

**PyTorch Dataset & DataLoader**<br>
reference: https://huggingface.co/transformers/custom_datasets.html#question-answering-with-squad-2-0

In [ ]:
# test_encodings = tokenizer(test_contexts, test_questions, truncation=True, max_length=512, padding=True)

In [ ]:
# test_encodingsを表示させようとするとmemory errorになるので注意
# print(type(test_encodings))
# print(len(test_encodings))

In [ ]:
# def add_token_positions(encodings, answers):
#     """
#     Args:
#         encodings: transformers.tokenization_utils_base.BatchEncoding
#         answers: List[Dict[str:int]]
#     Returns:
#         None
#     """
#     start_positions = []
#     end_positions = []
#     for i in range(len(answers)):
#         start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        
#         #### RIOW
#         #end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
#         end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

#         # if start position is None, the answer passage has been truncated
#         if start_positions[-1] is None:
#             start_positions[-1] = tokenizer.model_max_length
#         if end_positions[-1] is None:
#             end_positions[-1] = tokenizer.model_max_length
#         #### RIOWRIOW
    
#     encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

In [ ]:
# def add_token_ids(encodings, ids):
#     """
#     Args:
#         encodings: transformers.tokenization_utils_base.BatchEncoding
#         ids: List[str]
#     Returns:
#         None
#     """
#     #start_positions = []
#     #end_positions = []
#     ids_tokenized = []
#     for i, Id in enumerate(ids):
#         ids_tokenized.append(encodings.char_to_token(i, Id))

#         # if start position is None, the answer passage has been truncated
#         if ids_tokenized[-1] is None:
#             ids_tokenized[-1] = tokenizer.model_max_length
    
#     encodings.update({'id': ids_tokenized})

In [ ]:
# test_ids[:5]

In [ ]:
#add_token_positions(train_encodings, train_answers)
#add_token_ids(test_encodings, test_ids)
# reference: https://www.gitmemory.com/issue/huggingface/transformers/12201/862549850
# "Why not use tokenizerfast?"

In [ ]:
# class ChaiiDataset(Dataset):
#     def __init__(self, encodings):
#         self.encodings = encodings

#     def __getitem__(self, idx):
#         return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

#     def __len__(self):
#         return len(self.encodings.input_ids)

In [ ]:
# test_dataset = ChaiiDataset(test_encodings)
# test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)

In [ ]:
# for i, v in enumerate(test_dataloader):
#     print(i)
#     print(v)
#     print("input_ids shape:\n", v["input_ids"].shape)
#     print("token_type_ids shape:\n", v["token_type_ids"].shape)
#     print("attention_mask shape:\n", v["attention_mask"].shape)


# 0
# {'input_ids': tensor([[  101, 53459,   865,  ..., 14242, 13794,   102],
#         [  101, 10150, 10193,  ...,     0,     0,     0],
#         [  101,  1160, 31829,  ...,     0,     0,     0],
#         ...,
#         [  101,  1140, 27885,  ...,     0,     0,     0],
#         [  101,  1152, 27885,  ...,     0,     0,     0],
#         [  101,  1142, 17506,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 1, 1, 1],
#         [0, 0, 0,  ..., 0, 0, 0],
#         [0, 0, 0,  ..., 0, 0, 0],
#         ...,
#         [0, 0, 0,  ..., 0, 0, 0],
#         [0, 0, 0,  ..., 0, 0, 0],
#         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
#         [1, 1, 1,  ..., 0, 0, 0],
#         [1, 1, 1,  ..., 0, 0, 0],
#         ...,
#         [1, 1, 1,  ..., 0, 0, 0],
#         [1, 1, 1,  ..., 0, 0, 0],
#         [1, 1, 1,  ..., 0, 0, 0]])}
# input_ids shape:
#  torch.Size([8, 512])
# token_type_ids shape:
#  torch.Size([8, 512])
# attention_mask shape:
#  torch.Size([8, 512])
# 1
# {'input_ids': tensor([[   101,    893,  49978,  ...,  14242,  13794,    102],
#         [   101,   1140,  27885,  ...,      0,      0,      0],
#         [   101,   1144,  53500,  ...,      0,      0,      0],
#         ...,
#         [   101,   1154, 111312,  ...,      0,      0,      0],
#         [   101,   1140,  31484,  ...,      0,      0,      0],
#         [   101,  10212,  24962,  ...,      0,      0,      0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 1, 1, 1],
#         [0, 0, 0,  ..., 0, 0, 0],
#         [0, 0, 0,  ..., 0, 0, 0],
#         ...,
#         [0, 0, 0,  ..., 0, 0, 0],
#         [0, 0, 0,  ..., 0, 0, 0],
#         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
#         [1, 1, 1,  ..., 0, 0, 0],
#         [1, 1, 1,  ..., 0, 0, 0],
#         ...,
#         [1, 1, 1,  ..., 0, 0, 0],
#         [1, 1, 1,  ..., 0, 0, 0],
#         [1, 1, 1,  ..., 0, 0, 0]])}
# input_ids shape:
#  torch.Size([8, 512])
# token_type_ids shape:
#  torch.Size([8, 512])
# attention_mask shape:
#  torch.Size([8, 512])

Inference

In [ ]:
# Model
if DISABLE_INTERNET:
    model_path = "../input/localnb001-export-transformers"
    model = BertForQuestionAnswering.from_pretrained(model_path)
else:
    model = BertForQuestionAnswering.from_pretrained("bert-base-multilingual-cased")
    
# Optimizer
#optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
# Load model weights and optimizer state
output_model = "../input/localnb002-fine-tune/model.pth"
checkpoint = torch.load(output_model, map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
# # Scheduler
# num_epochs = 3
# num_training_steps = num_epochs * len(train_dataloader)
# lr_scheduler = get_scheduler(
#     "linear",
#     optimizer=optimizer,
#     num_warmup_steps=0,
#     num_training_steps=num_training_steps
# )

In [ ]:
from transformers import pipeline
model = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)
#device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#model.to(device)

In [ ]:
#len(test_dataloader)

In [ ]:
#test_ids, test_contexts, test_questions

In [ ]:
#test_contexts[:3]

In [ ]:
# found_sentences = [] #sentences to be ran through NER model later, one element for each file id
# ids = []

# for filename in test_dir.iterdir():
#     file_id = filename.stem
#     ids.append(file_id)
#     document_text = get_document_text(file_id, test=True)
#     sentences = split_into_sentences(document_text)
#     sentences = shorten_sentences(sentences)
    
#     file_sentences = []
#     for sentence in sentences:
#         for pattern, question in patterns.items():
#             file_sentences.append((sentence, question))
    
#     found_sentences.append(file_sentences)

In [ ]:
def read_chaii_test(df):
    """
    Args:
        df: pandas.DataFrame
    Returns:
        submission_df: pandas.DataFrame
    """
    ids = []
    #contexts = []
    #questions = []
    prediction_strings = []
    prediction_scores = []
    
    for i,row in df.iterrows():
        row_id = row["id"]
        row_context_sentences = row["context_sentences"]
        row_question = row["question"]
        
        for context_sentence in row_context_sentences:
            ids.append(row_id)
            output = model(question=row_question, context=context_sentence)
            prediction_strings.append(output["answer"])
            prediction_scores.append(output["score"])
            
    
    submission_df = pd.DataFrame({"id":ids, 
                              "PredictionString":prediction_strings,
                              "PredictionScore":prediction_scores})
    
    return submission_df

In [ ]:
submission_df = read_chaii_test(test)

In [ ]:
# prediction_strings = []
# prediction_scores = []
# for question, context in zip(test_questions, test_contexts):
#     output = model(question=question, context=context)
#     #### RIOW
# #     if output["score"] > 0.3:
# #          prediction_strings.append(output["answer"])
# #     else:
# #         prediction_strings.append("__dummyString__")
#     prediction_strings.append(output["answer"])
#     prediction_scores.append(output["score"])
#     #### RIOWRIOW

In [ ]:
# for Id, ps in zip(test_ids, prediction_strings):
#     print(Id, ps)

In [ ]:
# len(prediction_strings)

In [ ]:
# submission_df = pd.DataFrame({"id":test_ids, 
#                               "PredictionString":prediction_strings,
#                               "PredictionScore":prediction_scores})
#submission_df

In [ ]:
# Drop duplicates keeping PredictionString w/ the heighest PredictionScore
submission_df.sort_values(["id", "PredictionScore"], ascending=False, inplace=True)
submission_df.drop_duplicates("id", keep="first", inplace=True)
submission_df = submission_df[["id", "PredictionString"]]

In [ ]:
# Drop duplicates and leave only the most frequent value
#submission_df = submission_df.groupby("id")["PredictionString"].agg(lambda x: x.mode()[0])

In [ ]:
submission_df.to_csv("submission.csv", index=False)

In [ ]:
# progress_bar = tqdm(range(len(test_dataloader)))

# model.eval()
# pred_start_end_positions = []
# for batch in test_dataloader:
#     input_ids = batch['input_ids'].to(device)
#     attention_mask = batch['attention_mask'].to(device)
#     #start_positions = batch['start_positions'].to(device)
#     #end_positions = batch['end_positions'].to(device)
#     outputs = model(input_ids, attention_mask=attention_mask)#, start_positions=start_positions, end_positions=end_positions)
# #     loss = outputs[0]
# #     loss.backward()
# #     optimizer.step()
# #     lr_scheduler.step()
    
#     print("outputs.start_logits.shape:\n", outputs.start_logits.shape)
#     print("outputs.start_logits:\n", outputs.start_logits)
    
#     pred_start_position = torch.argmax(outputs.start_logits, dim=-1)
#     pred_end_position = torch.argmax(outputs.end_logits, dim=-1)
#     pred_start_end_positions.append((pred_start_position, pred_end_position))
#     #pred_string = input_ids[pred_start_positions:pred_end_positions]
#     #pred_string = test_encodings.token_to_chars(pred_ids)
#     #pred_strings.append(pred_string)
#     progress_bar.update(1)

In [ ]:
# def jaccard(str1, str2): 
#     a = set(str1.lower().split()) 
#     b = set(str2.lower().split())
#     c = a.intersection(b)
#     return float(len(c)) / (len(a) + len(b) - len(c))